# This notebook imports the raw seed points from 1_, and applies a decided upon Evaluation criteria

In [1]:
##Pandas and NumPy
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 8)
import numpy as np

##GeoSpatial imports
import geopandas as gpd
from geopy.distance import geodesic
import descartes as dc
import keplergl
from keplergl import KeplerGl

##for gemoetry to be compatible with geopandas
from shapely.wkt import load

## Swifter for applies
import dask.dataframe as dd
import multiprocessing
import copy
import swifter

## For distance calc test using columns
from math import sin, cos, sqrt, atan2, radians,pi
import gc
import datetime

##For map projections
import pyproj
from pyproj import Proj, transform, CRS

## manually setting proj directory bc I installed two
old = 'c:\\users\\rbijn006\\appdata\\local\\programs\\python\\python38\\lib\\site-packages\\pyproj\\proj_dir\\share\\proj'
new = 'c:\\users\\rbijn006\\appdata\\local\\programs\\python\\python38\\lib\\site-packages\\osgeo\\data\\proj'

pyproj.datadir.set_data_dir(old)


In [47]:
print(f'pandas=={pd.__version__}')
print(f'numpy=={np.__version__}')

pandas==1.0.5
numpy==1.19.2


In [2]:
## Function for getting accurate distances much faster than apply

## Now, write is as a function so multiple test points can be passed
## NEED COLUMN NAMES (LONGITUDE, LATITUDE, point_lat, point_lon) TO BE CONSISTENT

##############
## Function is auto set to calculate in miles, column names being the same as first 4 are required
## Specify dist=21,000,000 for the calculation to come out in feet (Earth's radius in feet)
##############

def distcalc(df, dist=3959):
    df['lat1']=(df['LATITUDE'])*pi/180
    df['lat2']=(df['point_lat'])*pi/180
    df['lon1']=(df['LONGITUDE'])*pi/180
    df['lon2']=(df['point_lon'])*pi/180
    
    df['x'] = (df['lon2'] - df['lon1']) * np.cos( 0.5*(df['lat2']+df['lat1']) )
    df['y'] = df['lat2'] - df['lat1']
    df['dist_to_pt']=dist * np.sqrt( df['x']*df['x'] + df['y']*df['y'] )
    df = df.drop(columns=['lat1', 'lat2','lon1','lon2','x','y'],axis=1)
    return df

In [3]:
## Filepaths to the raw seed points
uc = "results/raw_seedpoints/raw_seedpts_contusa.csv"
ak = "results/raw_seedpoints/raw_seedpts_ak.csv"
hi = "results/raw_seedpoints/raw_seedpts_hi.csv"

## Filepaths to RT & cxy data
r = "data/RT_OpenStores.csv"
c=  "data/CXY_ALL_1_11_21.csv"

In [4]:
## Read in seed pts (I am just combining here)
raw_seedpts = pd.concat([pd.read_csv(uc),pd.read_csv(ak),pd.read_csv(hi)], axis=0).reset_index(drop=True)
## Read in restaurant trends data
rt = pd.read_csv(r)
cxy = pd.read_csv(c)

c:\users\rbijn006\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (38,42,75) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
## Num of total seedpoints, len of concat should match

## Should return TRUE
43735+55+113 == len(raw_seedpts)

## If the program becomes problematically long ill split them back up!

True

# Data staging

## Start with ChainXY Staging

In [6]:
## Filtering out closed Cxy Locations
cxy = cxy[(cxy["Closed"]!="NO")]

## For now filtering out coming soon as well
cxy = cxy[(cxy["ComingSoon"]!="Yes")]

## Making City col all caps to match rt
cxy["City"] = cxy["City"].str.upper()

## Making lat/long all caps to play nice with distfunc

cxy.columns = ['StoreName', 'StoreNumber', 'ChainId', 'ChainName', 'ChainStatus',
       'Label', 'ParentChainId', 'ParentChainName', 'Address', 'Address2',
       'City', 'PostalCode', 'State', 'Country', 'PhoneNumber', 'Closed',
       'ComingSoon', 'StoreType', 'DistributorName', 'StoreHours',
       'OtherFields', 'AdditionalAttributes', 'LATITUDE', 'LONGITUDE',
       'GeoAccuracy', 'HashId', 'LastUpdate', 'FirstAppeared', 'LastSeen',
       'PrimaryCategory', 'Categories', 'SIC', 'NAICS', 'AdminLevel1Code',
       'AdminLevel1Name', 'AdminLevel1Label', 'AdminLevel2Code',
       'AdminLevel2Name', 'AdminLevel2Label', 'AdminLevel3Code',
       'AdminLevel3Name', 'AdminLevel3Label', 'AdminLevel4Code',
       'AdminLevel4Name', 'AdminLevel4Label', 'AdminLevel5Code',
       'AdminLevel5Name', 'AdminLevel5Label', 'AdminLevel6Code',
       'AdminLevel6Name', 'AdminLevel6Label']

### Need to start thinking here where do I see potential for improvement over current system, which uses a combination of QSR + Big Box within 0.5 miles

### FOUND Status Quo Big Box: Kohl's, BJ's, Costco, Meijer, Sam's Club, Lowes, Home Depot, Menard's, Kmart, Target, Walmart, Bass Pro, Orvis, IKEA, Best Buy

### Improvements: Include Large Hospital, large regional grocers (Publix, etc)

In [7]:
## Below is a list of what I looked through in the order I looked
## ['Department Store','Discount Stores','Grocery','Home Improvement + Building Supplies','Mass Merchant',
## 'Sports + Recreation','Furnishings','Computers + Consumer Electronics']

## Names of the stores in the list above as they appear in the cxy["ChainName"] column:
## "Kohl's" , "BJ's Wholesale Club",'Costco Wholesale', 'Meijer', "Sam's Club", "Lowe's", 'The Home Depot', "Menard's",
## 'Kmart', 'Target', 'Walmart USA', 'Bass Pro Shops', 'Orvis', 'IKEA', 'Best Buy'


tst_cat = 'Grocery'
print(np.sort(cxy[(cxy["PrimaryCategory"] == tst_cat)]["Categories"].unique()))
print(" ")
print(np.sort(cxy[(cxy["PrimaryCategory"] == tst_cat)]["ChainName"].unique()))


['Asian; Grocery' 'Big Box; Grocery' 'Big Box; Mass Merchant; Grocery'
 'Big Box; Mass Merchant; Grocery; Clothing + Apparel'
 'Clothing + Apparel; Big Box; Mass Merchant; Grocery'
 'Convenience Store; Grocery' 'Government Office; Grocery' 'Grocery'
 'Grocery; Big Box' 'Grocery; Convenience Store'
 'Grocery; Discount Stores; Mass Merchant'
 'Grocery; Drug Stores + Pharmacies' 'Grocery; Mass Merchant'
 'Grocery; Mexican' 'Grocery; Restaurant'
 'Mass Merchant; Big Box; Grocery' 'Mass Merchant; Grocery'
 'Mass Merchant; Grocery; Big Box' 'Mexican; Grocery'
 'Pet Health + Care; Grocery; Home Improvement + Building Supplies'
 'Restaurant - Juice Bars; Vitamins, Nutrition, and Supplements; Grocery'
 'Vitamins, Nutrition, and Supplements; Grocery']
 
['168 Market' '99 Ranch Market' 'AC Value Center' "AJ's Fine Foods" 'ALDI'
 'Acme Fresh Market' 'Acme Markets' 'Adams Hometown Market'
 "Akin's Natural Food Market" 'Albertsons' 'Albertsons Market' 'Amazon Go'
 'Amigos (United)' "Andronico's Comm

In [8]:
## Filter on sub-categories on our test cat
np.sort(cxy[(cxy["PrimaryCategory"] == tst_cat)&(cxy["Categories"].isin(['Big Box; Grocery', 'Big Box; Mass Merchant; Grocery',
 'Big Box; Mass Merchant; Grocery; Clothing + Apparel',
 'Clothing + Apparel; Big Box; Mass Merchant; Grocery',
 'Grocery; Big Box','Grocery; Discount Stores; Mass Merchant','Mass Merchant; Big Box; Grocery', 'Mass Merchant; Grocery'
 'Mass Merchant; Grocery; Big Box']))]["ChainName"].unique())

array(['ALDI', 'Albertsons', "BJ's Wholesale Club", 'Costco Wholesale',
       'Food 4 Less', 'Food 4 Less (California)',
       "Fry's Food & Drug Stores", 'Kroger', "Lowe's Foods",
       "Lowe's Market", 'Meijer', 'Publix Supermarkets', 'Safeway',
       "Sam's Club", 'Save-A-Lot (Onex)', 'Save-A-Lot (Pyramid Foods)',
       'Super Dollar Discount Foods', "Trader Joe's",
       'Whole Foods Market'], dtype=object)

In [9]:
## 158k RT all PROPORTIONALLY WOULD REDUCE TO 105k
## 65k RT DT no captive, reduces to  43k

##the only way we are eliminating is if they do not follow the criteria 

In [10]:
## Isolating status quo (Tetrad) Big Boxes

big_box = cxy[(cxy["ChainName"].isin(["Kohl's","BJ's Wholesale Club",'Costco Wholesale','Meijer',"Sam's Club","Lowe's",
                            'The Home Depot', "Menard's",'Kmart', 'Target', 'Walmart USA', 'Bass Pro Shops', 
                            'Orvis', 'IKEA', 'Best Buy']))][['StoreNumber', 'ChainId', 'ChainName', 'City', 
 'State','StoreType','LATITUDE', 'LONGITUDE','HashId']]

## Get rid of some pickup only Walmarts
big_box = big_box[(big_box["StoreType"]!='Pickup only')]

In [11]:
## Isolating QSRs, there are a lot of them could potentially cut this down
## Another note is could sub this for RT data to exclude food courts

qsr = cxy[(cxy["PrimaryCategory"]=='Restaurant - QSR/Fast Food')][['StoreNumber', 'ChainId', 'ChainName','City', 
 'State','StoreType','LATITUDE', 'LONGITUDE','HashId']]

In [12]:
## Setting up the labels I will grpby on when in the loop
qsr["LABEL"] = "QSR"
big_box["LABEL"] = "BBOX"

In [13]:
## Deleting CXY from memory since it is so large
del cxy

## Staging for RT - can use RT QSR instead of CXY

In [14]:
## Isolate the QSR segment of the rt data, which is 98% of the data
print(rt["SEGMENT"].unique())
print(len(rt[(rt["SEGMENT"]=="QSR")])/len(rt))

rt = rt[(rt["SEGMENT"]=="QSR")]

['QSR' 'FSR']
0.9855522395609398


In [15]:
## Next, exclude Popeyes. INCLUDING to get the distance to PLK as an output. 

#rt = rt[~(rt["CHAIN"]=="Popeyes")]

In [16]:
## Excluding captive locations for now. More than likely will not make a huge difference we just wont see seed points in malls
## or in millitary bases. Very easy to change this.

In [17]:
## Creating list of LOCATION_DESCRIPTION that qualify as captive

captive_locations = ['Airport Terminal','Casino-Gaming','Office Building','Military Base','Hospital/Medical Facility',
                    'College-University-Trade School','Railroad Terminal','Mall-Enclosed', 'Amusement-Theme Park',
                       'Arena-Stadium', 'Movie Theater', 'Recreational Sports Ctr (include Ice/Roller Rink)''Bus Terminal', 
                     'Zoo','Small Enclosed Mall','Museum','Inside Something Undefined/Misc Retail', 'School (K-12)', 
                     'Ferry Terminal']

#rt["LOCATION_DESCRIPTION"].unique()

In [18]:
print(f'Removing Captive Locations by Location Code Removes {len(rt)-len(rt[~(rt["LOCATION_DESCRIPTION"].isin(captive_locations))])} sites, or {round(100*(len(rt)-len(rt[~(rt["LOCATION_DESCRIPTION"].isin(captive_locations))]))/len(rt),2)}% of all RT QSR rows')

Removing Captive Locations by Location Code Removes 2870 sites, or 1.84% of all RT QSR rows


In [19]:
## Filtering out captive locations
rt = rt[~(rt["LOCATION_DESCRIPTION"].isin(captive_locations))]

In [20]:
## WARNING!!!!! DO NOT DO THIS b/c facility codes that have shoping center in them also refer to pad sites. I pre-filtered captive locations
## Trying to see if I can filter out shopping center and Airport


#searchfor = ["AIRPORT", "SHOPPING CENTER"]
#filters = rt[~(rt["LOCATION_DESCRIPTION"].isin(captive_locations)) & ~(rt["FACILITY_NAME"].isna())]
#filters = filters[(filters['FACILITY_NAME'].str.contains('|'.join(searchfor)))]
#filters

In [21]:
## Cutting down rt to mimic the ChainXY data. Will Change column names as well
rt = rt[['STORE_NO','CHAIN_NO','CHAIN','GEOCITY', 'GEOSTATE','SERVICE_CODE','LATITUDE', 'LONGITUDE']]
rt["HashId"] = pd.NA
rt["LABEL"] = "QSR"

## Changing columns to mimic ChainXY
rt.columns = ['StoreNumber', 'ChainId', 'ChainName', 'City', 'State', 'StoreType',
       'LATITUDE', 'LONGITUDE', 'HashId', 'LABEL']

In [22]:
## Cutting to dt only to mirror Tetrad. This can easily be changed
rt
rt = rt[(rt["StoreType"]=="DT")]
rt

,StoreNumber,ChainId,ChainName,City,State,StoreType,LATITUDE,LONGITUDE,HashId,LABEL
2,468006,20,A & W,WINNECONNE,WI,DT,44.111902,-88.657309,<NA>,QSR
3,467999,20,A & W,MILES CITY,MT,DT,46.402699,-105.822015,<NA>,QSR
8,468134,8541,Amigo's,OMAHA,NE,DT,41.232437,-96.119234,<NA>,QSR
16,468374,24,Arby's,NEW BRIGHTON,PA,DT,40.738581,-80.316655,<NA>,QSR
...,...,...,...,...,...,...,...,...,...,...
158142,467749,4,Dunkin' Donuts,VISALIA,CA,DT,36.312855,-119.313629,<NA>,QSR
158144,467751,4,Dunkin' Donuts,VIRGINIA BEACH,VA,DT,36.840402,-76.094753,<NA>,QSR
158145,467752,4,Dunkin' Donuts,HARRISONBURG,VA,DT,38.419521,-78.869827,<NA>,QSR
158155,467839,10,Long John Silver's,FORESTVILLE,MD,DT,38.845196,-76.884308,<NA>,QSR


## Finished Staging ChainXY & RT - Combine the data you want to use to make your "Evaluation Table" here

### NOTE: CXY QSRs: 142,600   RT QSRs: 150,621 (with captive loc removed) So pretty close!

In [23]:
## I am using CXY big box & RT Drive Thru only for the first pass just to see what happens
eval_tbl = pd.concat([rt,big_box], axis=0).reset_index(drop=True)

## Should return TRUE when using big_box & RT
len(eval_tbl) == len(big_box) + len(rt)

True

In [24]:
## 77,575 total rows. Easy.
#eval_tbl

In [25]:
## Number of Popeyes left in eval_table
len(eval_tbl[(eval_tbl["ChainName"]=="Popeyes")])

2027

# Now start writing code to calculate dist for each raw seed point

In [26]:
#####################################################################
### All that is required of the eval table is it has two columns named LATITUDE & LONGITUDE
###
###
########################################################################

def evaluation_criteria(lat,lon,eval_tbl):
    #Set lat lon to be evaluated
    eval_tbl["point_lat"] = lat
    eval_tbl["point_lon"] = lon
    
    #Perform distcalc 
    eval_tbl = eval_tbl.pipe(distcalc)
    
    #Get & return counts
    ct_QSR_halfmi  = sum(eval_tbl[(eval_tbl["dist_to_pt"]<=0.5)]["LABEL"]=="QSR")
    ct_BBOX_halfmi = sum(eval_tbl[(eval_tbl["dist_to_pt"]<=0.5)]["LABEL"]=="BBOX")
    ct_plk_tenthmi = sum(eval_tbl[(eval_tbl["dist_to_pt"]<=0.1)]["ChainName"]=="Popeyes")
    ct_plk_qmi = sum(eval_tbl[(eval_tbl["dist_to_pt"]<=0.25)]["ChainName"]=="Popeyes")
    
    return ct_QSR_halfmi,ct_BBOX_halfmi,ct_plk_tenthmi,ct_plk_qmi
    

In [27]:
## Test, returns a tuple
evaluation_criteria(raw_seedpts["seed_LAT"][7],raw_seedpts["seed_LON"][7], eval_tbl)

(13, 2, 0, 1)

In [37]:
## This is the loop that calculates with no swifter. Clocked it at 14m 57s for 43,903 rows
raw_seedpts["results"] = raw_seedpts.apply(lambda row: evaluation_criteria(row["seed_LAT"],row["seed_LON"], eval_tbl), axis=1)

## This is the loop that calculates with swifter. Good old swifter defaulting to Pandas apply again.
#raw_seedpts["results"] = raw_seedpts.swifter.apply(lambda row: evaluation_criteria(row["seed_LAT"],row["seed_LON"], eval_tbl), axis=1)

## Forcing the use of dask for the apply here. Clocked it at 6m 45s for 43,903 row
raw_seedpts["results"]  = dd.from_pandas(raw_seedpts, npartitions=2*multiprocessing.cpu_count())\
        .map_partitions(lambda df: df.apply(lambda row: evaluation_criteria(row["seed_LAT"],row["seed_LON"], eval_tbl), axis=1))\
        .compute(scheduler='processes')

In [33]:
## Columns to fill in counts on
raw_seedpts["ct_QSR_0.5mi"] = raw_seedpts["results"].apply(lambda row: row[0])
raw_seedpts["ct_BB_0.5mi"] = raw_seedpts["results"].apply(lambda row: row[1])
raw_seedpts["ct_plk_tenthmi"] = raw_seedpts["results"].apply(lambda row: row[2])
raw_seedpts["ct_plk_qmi"] = raw_seedpts["results"].apply(lambda row: row[3])

In [35]:
import copy
#check_check = copy.deepcopy(raw_seedpts)

In [36]:
## check check are the dask results
check_check

,geometry,area,seed_LAT,seed_LON,STATE,results
0,POLYGON ((-83.94854614356376 38.07321482506316...,199653.082246,38.075029,-83.949671,CONT_US,"(10, 0, 0, 0)"
1,POLYGON ((-111.9693898853349 41.24347261312843...,198739.095562,41.244639,-111.972855,CONT_US,"(14, 1, 0, 1)"
2,POLYGON ((-117.0814745048376 34.84804484478374...,181049.745578,34.850887,-117.082879,CONT_US,"(10, 0, 0, 0)"
3,POLYGON ((-84.53777046800231 33.56778750202658...,180358.265807,33.569272,-84.540125,CONT_US,"(11, 1, 0, 1)"
...,...,...,...,...,...,...
43899,POLYGON ((-155.0658403191121 19.69471168779418...,26605.490282,19.694718,-155.066716,HI,"(5, 1, 0, 0)"
43900,POLYGON ((-155.0653773354194 19.69017768633985...,26605.490282,19.690184,-155.066253,HI,"(3, 0, 0, 0)"
43901,POLYGON ((-155.0383895809582 19.62203959911077...,26605.490282,19.622046,-155.039265,HI,"(1, 0, 0, 0)"
43902,POLYGON ((-159.3669130581232 21.98104676977263...,26605.490282,21.981039,-159.367797,HI,"(2, 1, 0, 0)"


In [39]:
## Wonderful dask is the same as pandas and takes 1/2 the time. Why wont swifter recognize this?
sum(raw_seedpts["results"] == check_check["results"])

43903

In [150]:
raw_seedpts.loc[3333]

geometry        POLYGON ((-84.30777825925239 38.3804861887164,...
area                                                      52888.6
seed_LAT                                                  38.3811
seed_LON                                                 -84.3078
STATE                                                     CONT_US
ct_QSR_0.5mi                                                    5
ct_BB_0.5mi                                                     1
results                                                    (5, 1)
Name: 3333, dtype: object

In [38]:
## MANUALLY PUT IN INDEX TO VALIDATE HERE
## Not getting copy warning here bc setting of points comes from outside data frame (raw_seedpts)

idx= 1
eval_tbl["point_lat"] = raw_seedpts["seed_LAT"][idx]
eval_tbl["point_lon"] = raw_seedpts["seed_LON"][idx]

#Perform distcalc
eval_tbl = eval_tbl.pipe(distcalc)

## Point being calculated in miles (0.5mi is same as Tetrad), look at tenth mile as well, or other dist
#eval_tbl[(eval_tbl["dist_to_pt"]<=0.5)]

,StoreNumber,ChainId,ChainName,City,State,StoreType,LATITUDE,LONGITUDE,HashId,LABEL,point_lat,point_lon,dist_to_pt
13476,84685,11,McDonald's-TR,OGDEN,UT,DT,41.245640,-111.970490,<NA>,QSR,41.244639,-111.972855,0.140994
19339,118297,15,Taco Bell-TR,OGDEN,UT,DT,41.244017,-111.970954,<NA>,QSR,41.244639,-111.972855,0.107723
25523,147748,216,Del Taco,OGDEN,UT,DT,41.245811,-111.977546,<NA>,QSR,41.244639,-111.972855,0.256810
25833,175559,9,KFC-TR,OGDEN,UT,DT,41.243895,-111.972226,<NA>,QSR,41.244639,-111.972855,0.060935
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56809,420990,552,Popeyes,OGDEN,UT,DT,41.244197,-111.975185,<NA>,QSR,41.244639,-111.972855,0.124850
57705,426620,6609,Freddy's,OGDEN,UT,DT,41.244193,-111.974621,<NA>,QSR,41.244639,-111.972855,0.096793
59034,435157,648,Sonic,OGDEN,UT,DT,41.243336,-111.970474,<NA>,QSR,41.244639,-111.972855,0.153015
76724,4411,4818,The Home Depot,OGDEN,UT,NaN,41.247217,-111.975392,4818-f542c2f17d928155ef5ac5356e44ca12-7945a8cd...,BBOX,41.244639,-111.972855,0.221558


In [39]:
print("INDEX: ", idx)
print("QSR_hlfmi: ", sum(eval_tbl[(eval_tbl["dist_to_pt"]<=0.5)]["LABEL"]=="QSR"))
print("BBOX_hlfmi: ", sum(eval_tbl[(eval_tbl["dist_to_pt"]<=0.5)]["LABEL"]=="BBOX"))
print("plk_tenthmi: ", sum(eval_tbl[(eval_tbl["dist_to_pt"]<=0.1)]["ChainName"]=="Popeyes"))
print("plk_qmi: ", sum(eval_tbl[(eval_tbl["dist_to_pt"]<=0.25)]["ChainName"]=="Popeyes"))
print("plk_hlfmi: ", sum(eval_tbl[(eval_tbl["dist_to_pt"]<=0.5)]["ChainName"]=="Popeyes"))
print("plk_1mi: ", sum(eval_tbl[(eval_tbl["dist_to_pt"]<=1)]["ChainName"]=="Popeyes"))

INDEX:  1
QSR_hlfmi:  14
BBOX_hlfmi:  1
plk_hlfmi:  1
plk_1mi:  1


## Test Filter based on PLK

In [88]:
raw_seedpts[(raw_seedpts['plk_tenthmi']==0)&((raw_seedpts['ct_QSR_0.5mi']>=3) | ((raw_seedpts['ct_QSR_0.5mi']>=1)&(raw_seedpts['ct_BB_0.5mi']>=1)))]

KeyError: 'ct_plk_halfmi'

## End computation. Now see which seeds passed

In [87]:
## Criteria was (>=3 QSRs), OR (>=1QSR + >=1BBOX)
final_seedpoints = raw_seedpts[((raw_seedpts['ct_QSR_0.5mi']>=3) | ((raw_seedpts['ct_QSR_0.5mi']>=1)&(raw_seedpts['ct_BB_0.5mi']>=1)))]

KeyError: 'ct_QSR_0.5mi'

# 26,541 make the cut wit hno PLK filter. With halfmi PLK filter, 23,309

# Tetrad had ~21,871 or 23,943 make the cut in a file I have

In [86]:
final_seedpoints[(final_seedpoints["STATE"]=="HI")].shape

NameError: name 'final_seedpoints' is not defined

In [144]:
final_seedpoints[(final_seedpoints["STATE"]=="AK")].shape

(36, 8)

In [148]:
final_seedpoints[(final_seedpoints["STATE"]=="CONT_US")].shape

(26445, 8)

## Add in State & City in loop after!!!! To get that info for all seed points!!!!

In [195]:
final_seedpoints

,geometry,area,seed_LAT,seed_LON,STATE,ct_QSR_0.5mi,ct_BB_0.5mi,results
0,POLYGON ((-83.94854614356376 38.07321482506316...,199653.082246,38.075029,-83.949671,CONT_US,10,0,"(10, 0)"
1,POLYGON ((-111.9693898853349 41.24347261312843...,198739.095562,41.244639,-111.972855,CONT_US,13,1,"(13, 1)"
2,POLYGON ((-117.0814745048376 34.84804484478374...,181049.745578,34.850887,-117.082879,CONT_US,10,0,"(10, 0)"
3,POLYGON ((-84.53777046800231 33.56778750202658...,180358.265807,33.569272,-84.540125,CONT_US,10,1,"(10, 1)"
...,...,...,...,...,...,...,...,...
43898,POLYGON ((-155.0624683096572 19.69736667675373...,26605.490282,19.697373,-155.063344,HI,3,3,"(3, 3)"
43899,POLYGON ((-155.0658403191121 19.69471168779418...,26605.490282,19.694718,-155.066716,HI,5,1,"(5, 1)"
43900,POLYGON ((-155.0653773354194 19.69017768633985...,26605.490282,19.690184,-155.066253,HI,3,0,"(3, 0)"
43902,POLYGON ((-159.3669130581232 21.98104676977263...,26605.490282,21.981039,-159.367797,HI,2,1,"(2, 1)"


In [221]:
## Testing point geocoder
test = pd.read_excel("results/test_excel.xlsx").drop("geometry", axis=1)

In [222]:
test.dtypes

area            float64
seed_LAT        float64
seed_LON        float64
STATE            object
ct_QSR_0.5mi      int64
dtype: object

In [223]:
## Convert to GPD
test = gpd.GeoDataFrame(test,crs = "EPSG:4326" , geometry= gpd.points_from_xy(test.seed_LON, test.seed_LAT))
test.dtypes

area             float64
seed_LAT         float64
seed_LON         float64
STATE             object
ct_QSR_0.5mi       int64
geometry        geometry
dtype: object

In [225]:
test["geometry"]

0      POINT (-83.94967 38.07503)
1     POINT (-111.97286 41.24464)
2     POINT (-117.08288 34.85089)
3      POINT (-84.54013 33.56927)
                 ...             
96     POINT (-78.77567 35.59218)
97     POINT (-83.75777 31.96115)
98     POINT (-97.76035 35.49027)
99     POINT (-83.76255 34.59352)
Name: geometry, Length: 100, dtype: geometry

In [ ]:
gpd.tools.reverse_geocode(test["geometry"])

In [209]:
## Going from excel to .shp for AY
test = pd.read_excel("results/test_excel.xlsx")

In [210]:
test.dtypes

geometry         object
area            float64
seed_LAT        float64
seed_LON        float64
STATE            object
ct_QSR_0.5mi      int64
dtype: object

In [211]:
#for gemoetry to be compatible with geopandas
from shapely.wkt import loads
test["geometry"] = test["geometry"].apply(lambda row: loads(row))

In [212]:
test = gpd.GeoDataFrame(test,crs = "EPSG:4326" , geometry= test["geometry"])

In [213]:
test.dtypes

geometry        geometry
area             float64
seed_LAT         float64
seed_LON         float64
STATE             object
ct_QSR_0.5mi       int64
dtype: object

In [190]:
test.to_file("results/test_file.shp", index=False)

In [193]:
test = gpd.read_file("results/test_file.shp")